In [2]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np


# getting info from sports reference

In [3]:
def get_winners (link):
    print (link)
    page = requests.get(link)
    #print('here')
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    i=0
    k=0
    winners=[]
    for thing in list(range(1,100)):
        if k == 22:
            break
        elif k == 0 or k== 1 or k==2 or k==3 or k==5 or k==6 or k==7 or k==8 or k==10 or k==11 or k==12 or k==13 or k==15 or k==16 or k==17 or k==18 or k==20 or k==21:
            try:
                winners.append(idk[k].select('div')[i].select('div[class="winner"]')[0].select('span')[0].string)
                i+=3
            except:
                i=0
                k+=1
                continue
        else:
            i=0
            k+=1
            continue
    winners_int=[]
    for thing in winners:
        winners_int.append(int(thing))
    return winners_int

In [4]:
#function for getting the teams
#gets the teams that played each other all together
def get_teams (link,k):
    page = requests.get(link)
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    i=1
    maybe = []
    for thing in list(range(1,100)):
        if i%3 !=0:
            try:
                maybe.append(idk[k].select('div')[i].select('span')[0].string)
                i+=1
            except:
                i+=1
                continue
        else:
            i+=1
            continue
    return maybe



In [5]:
#uses the function to split up the teams that played each other into 2 separate lists
def split_up_teams (link):
    All_teams = []
    for thing in list(range(0,23)):
        new=get_teams (link,thing)
        All_teams.extend(new)
    All_teams.pop(-1)
    All_teams.pop(30)
    All_teams.pop(60)
    All_teams.pop(90)
    All_teams.pop(120)
    first_team = All_teams[0::2]
    second_team = All_teams[1::2]
    first_team_int = []
    for thing in first_team:
        first_team_int.append(int(thing))
    second_team_int = []
    for thing in second_team:
        second_team_int.append(int(thing))
    first_team_pd = pd.DataFrame(first_team_int)
    second_team_pd = pd.DataFrame(second_team_int)
    point_difference = first_team_pd-second_team_pd
    point_difference_list=point_difference.values.tolist()
    point_diff_abs = []
    for thing in point_difference_list:
        point_diff_abs.append(abs(thing[0]))
    return first_team_int,second_team_int,point_diff_abs



In [6]:
def get_team_names (link,k):
    page = requests.get(link)
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    maybe = []
    for thing in list(range(0,100)):
        try:
            maybe.append(idk[k].select('a[href^=/cbb/schools]')[thing].string)
        except:
            continue
    return maybe



In [7]:
def get_teams_get (link):
    team_names = []
    for thing in list(range(0,23)):
        new=get_team_names (link,thing)
        team_names.extend(new)
    team_names.pop(-1)
    team_names.pop(30)
    team_names.pop(60)
    team_names.pop(90)
    team_names.pop(120)
    return team_names



In [8]:
def split_up_teams_2 (link):
    All_teams = []
    for thing in list(range(0,23)):
        new=get_teams (link,thing)
        All_teams.extend(new)
    All_teams.pop(-1)
    All_teams.pop(30)
    All_teams.pop(60)
    All_teams.pop(90)
    All_teams.pop(120)
    return All_teams

In [9]:
def everything (link):
    first_team_int = []
    second_team_int = []
    point_diff_abs = []
    winners_int = []

    winners=get_winners (link)
    three_lists=split_up_teams (link)
    first_team_int.extend(three_lists[0])
    second_team_int.extend(three_lists[1])
    point_diff_abs.extend(three_lists[2])
    winners_int.extend(winners)
    #time.sleep(3)
    #print(len(first_team_int), len(second_team_int),len(point_diff_abs), len(winners_int))
    df = pd.DataFrame({"first_team": first_team_int, "second_team": second_team_int,
                       "winner" : winners_int, "point_difference": point_diff_abs})
    result = []
    i = 0
    for thing in list(df['winner']):
        #print(f'this is {i}')
        if list(df['first_team'])[i] > list(df['second_team'])[i] and list(df['first_team'])[i] == list(df['winner'])[i]:
            #print(f'this is if1')
            result.append(2)
        elif list(df['second_team'])[i] > list(df['first_team'])[i] and list(df['second_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif1')
            result.append(2)
        elif list(df['first_team'])[i] < list(df['second_team'])[i] and list(df['first_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif2')
            result.append(1)
        elif list(df['second_team'])[i] < list(df['first_team'])[i] and list(df['second_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif3')
            result.append(1)
        else:
            result.append(0)
        i+=1
    df['result']=result
    newdf2 = pd.DataFrame(np.repeat(df.values,2,axis=0))
    newdf2.columns = df.columns
    team_names=get_teams_get (link)
    All_teams=split_up_teams_2(link)
    rankings= []
    for thing in All_teams:
        #print(thing)
        rankings.append(int(thing))
    newdf2['team_name'] = team_names
    newdf2['team_rank'] = rankings
    win = []
    i = 0
    for thing in list(newdf2['team_rank']):
        if list(newdf2['first_team'])[i] == list(newdf2['second_team'])[i]:
            win.append(9999)
        elif list(newdf2['team_rank'])[i] == list(newdf2['winner'])[i]:
            win.append(1)
        elif list(newdf2['team_rank'])[i] != list(newdf2['winner'])[i]:
            win.append(0)
        i+=1
    newdf2['win']=win
    #if the team has a lower seed number, it is given '1', higher seed number '0'
    higher_or_lower_seed = []
    i = 0
    for thing in list(newdf2['team_rank']):
        if list(newdf2['first_team'])[i] == list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(9999)
        elif list(newdf2['team_rank'])[i] == list(newdf2['first_team'])[i] and list(newdf2['first_team'])[i] < list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(1)
        elif list(newdf2['team_rank'])[i] == list(newdf2['first_team'])[i] and list(newdf2['first_team'])[i] > list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(0)
        elif list(newdf2['team_rank'])[i] == list(newdf2['second_team'])[i] and list(newdf2['second_team'])[i] < list(newdf2['first_team'])[i]:
            higher_or_lower_seed.append(1)
        elif list(newdf2['team_rank'])[i] == list(newdf2['second_team'])[i] and list(newdf2['second_team'])[i] > list(newdf2['first_team'])[i]:
            higher_or_lower_seed.append(0)
        i+=1
    newdf2['higher_or_lower_seed']=higher_or_lower_seed
    #newdf2.to_csv(f'{link}')
    newdf2.rename(columns = {'team_name':'name'}, inplace = True)
    newdf2['index']=list(range(0,126))
    return newdf2

# getting score info from ESPN

In [10]:
def get_score_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
        
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/321')
    
    names = []
    PTS = []
    FG_perc = []
    _3p_perc = []
    FT_perc = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    PTS.append(float(thing.select('td')[3].string))
                    FG_perc.append(float(thing.select('td')[5].string))
                    _3p_perc.append(float(thing.select('td')[7].string))
                    FT_perc.append(float(thing.select('td')[9].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    PTS.append(float(idk2[i].select('td')[3].string))
                    FG_perc.append(float(idk2[i].select('td')[5].string))
                    _3p_perc.append(float(idk2[i].select('td')[7].string))
                    FT_perc.append(float(idk2[i].select('td')[9].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    PTS.append(float(thing.select('td')[3].string))
                    FG_perc.append(float(thing.select('td')[5].string))
                    _3p_perc.append(float(thing.select('td')[7].string))
                    FT_perc.append(float(thing.select('td')[9].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    PTS.append(float(idk[i].select('td')[3].string))
                    FG_perc.append(float(idk[i].select('td')[5].string))
                    _3p_perc.append(float(idk[i].select('td')[7].string))
                    FT_perc.append(float(idk[i].select('td')[9].string))
                    i+=1
                except:
                    continue
                    i+=1
    D = {'Name':names, 'PTS':PTS, 'FG_perc':FG_perc, '_3p_perc':_3p_perc, 'FT_perc':FT_perc}
    df2 = pd.DataFrame(data=D)
    
    the_year = []
    for thing in df2['Name']:
        the_year.append(year)
        
    df2['year']=the_year
            


            
    return df2

# ESPN Rebound Data

In [11]:
def get_rebound_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
        
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/321')
    
    names = []
    ORPG = []
    DRPG = []
    RPG = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    ORPG.append(float(thing.select('td')[4].string))
                    DRPG.append(float(thing.select('td')[6].string))
                    RPG.append(float(thing.select('td')[8].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    ORPG.append(float(idk2[i].select('td')[4].string))
                    DRPG.append(float(idk2[i].select('td')[6].string))
                    RPG.append(float(idk2[i].select('td')[8].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    ORPG.append(float(thing.select('td')[4].string))
                    DRPG.append(float(thing.select('td')[6].string))
                    RPG.append(float(thing.select('td')[8].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    ORPG.append(float(idk[i].select('td')[4].string))
                    DRPG.append(float(idk[i].select('td')[6].string))
                    RPG.append(float(idk[i].select('td')[8].string))
                    i+=1
                except:
                    continue
                    i+=1
                    
    D = {'Name':names, 'ORPG':ORPG, 'DRPG':DRPG, 'RPG':RPG}
    df2 = pd.DataFrame(data=D)

            
            


            
    return df2

# ESPN Field Goals

In [12]:
def get_field_goal_info (year):
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/321')
        
    names = []
    _2pm = []
    _2pa = []
    _2p_perc = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    _2pm.append(float(thing.select('td')[9].string))
                    _2pa.append(float(thing.select('td')[10].string))
                    _2p_perc.append(float(thing.select('td')[11].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    _2pm.append(float(idk2[i].select('td')[9].string))
                    _2pa.append(float(idk2[i].select('td')[10].string))
                    _2p_perc.append(float(idk2[i].select('td')[11].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    _2pm.append(float(thing.select('td')[9].string))
                    _2pa.append(float(thing.select('td')[10].string))
                    _2p_perc.append(float(thing.select('td')[11].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    _2pm.append(float(idk[i].select('td')[9].string))
                    _2pa.append(float(idk[i].select('td')[10].string))
                    _2p_perc.append(float(idk[i].select('td')[11].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, '_2pm':_2pm, '_2pa':_2pa, '_2p_perc':_2p_perc}
    df2 = pd.DataFrame(data=D)

   


            
    return df2

# ESPN 3-point data

In [13]:
def get_3_point_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    _3pm = []
    _3pa = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    _3pm.append(float(thing.select('td')[4].string))
                    _3pa.append(float(thing.select('td')[5].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    _3pm.append(float(idk2[i].select('td')[4].string))
                    _3pa.append(float(idk2[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    _3pm.append(float(thing.select('td')[4].string))
                    _3pa.append(float(thing.select('td')[5].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    _3pm.append(float(idk[i].select('td')[4].string))
                    _3pa.append(float(idk[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, '_3pm':_3pm, '_3pa':_3pa}
    df2 = pd.DataFrame(data=D)

            
    return df2

# ESPN assists

In [14]:
def get_assists_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    APG = []
    AST_TO = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    APG.append(float(thing.select('td')[4].string))
                    AST_TO.append(float(thing.select('td')[7].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    APG.append(float(idk2[i].select('td')[4].string))
                    AST_TO.append(float(idk2[i].select('td')[7].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    APG.append(float(thing.select('td')[4].string))
                    AST_TO.append(float(thing.select('td')[7].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    APG.append(float(idk[i].select('td')[4].string))
                    AST_TO.append(float(idk[i].select('td')[7].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, 'APG':APG, 'AST_TO':AST_TO}
    df2 = pd.DataFrame(data=D)

            
    return df2

# ESPN Steals info

In [15]:
def get_steals_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    STPG = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    STPG.append(float(thing.select('td')[4].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    STPG.append(float(idk2[i].select('td')[4].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    STPG.append(float(thing.select('td')[4].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    STPG.append(float(idk[i].select('td')[4].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, 'STPG':STPG}
    df2 = pd.DataFrame(data=D)

            
    return df2

# ESPN get blocks

In [16]:
def get_blocks_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    BLKPG = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    BLKPG.append(float(thing.select('td')[5].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    BLKPG.append(float(idk2[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    BLKPG.append(float(thing.select('td')[5].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    BLKPG.append(float(idk[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, 'BLKPG':BLKPG}
    df2 = pd.DataFrame(data=D)

            
    return df2

# Everything for ESPN - Merges everything also

In [17]:
def merge_espn (year):
    df3 = get_score_info (year)
    df4 = get_rebound_info (year)
    df5 = get_field_goal_info (year)
    df6=get_3_point_info (year)
    df7=get_assists_info (year)
    df8=get_steals_info (year)
    df9=get_blocks_info (year)
    df2=pd.merge(df3,df4, how = 'inner', on = 'Name')
    df2=pd.merge(df5,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df6,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df7,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df8,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df9,df2, how = 'inner', on = 'Name')
    return df2

# Merging espn and sports reference

In [18]:
#need to run this
def find_missing_values (blah):
    the_values = []
    for thing in list(range(0,126)):
        if thing in blah:
            continue
        else:
            #print(thing)
            the_values.append(thing)
    return the_values


In [19]:
#still working on this - dont use
def add_to_df (name1, num,df, df2, newdf2):
    maybe=df2[(df2['Name']==f'{name1}')]
    maybe2=newdf2[(newdf2['index']==num)]
    dict = {'first_team':list(maybe2['first_team'])[0],'second_team':list(maybe2['second_team'])[0],
            'winner':list(maybe2['winner'])[0],
            'point_difference':list(maybe2['point_difference'])[0],'result':list(maybe2['result'])[0],
            'name':list(maybe2['name'])[0],
            'team_rank':list(maybe2['team_rank'])[0],'win':list(maybe2['win'])[0],
            'higher_or_lower_seed':list(maybe2['higher_or_lower_seed'])[0],
            'index':list(maybe2['index'])[0],'year_x':list(maybe['year'])[0], 'Name': list(maybe['Name'])[0],
            'PTS':list(maybe['PTS'])[0],'year_y':list(maybe['year'])[0],
            'FG_perc':list(maybe['FG_perc'])[0],'_3p_perc':list(maybe['_3p_perc'])[0],
            'FT_perc':list(maybe['FT_perc'])[0],'year_y':list(maybe['year'])[0],
            'BLKPG':list(maybe['BLKPG'])[0],'STPG':list(maybe['STPG'])[0],
            'APG':list(maybe['APG'])[0],'AST_TO':list(maybe['AST_TO'])[0],
            '_3pm':list(maybe['_3pm'])[0],'_3pa':list(maybe['_3pa'])[0],
            '_2pm':list(maybe['_2pm'])[0],'_2pa':list(maybe['_2pa'])[0],
            '_2p_perc':list(maybe['_2p_perc'])[0],'ORPG':list(maybe['ORPG'])[0],
            'DRPG':list(maybe['DRPG'])[0],'RPG':list(maybe['RPG'])[0],}
    possibly2=df.append(dict, ignore_index=True)
    return possibly2

# Making dictionary to reference sports reference and ESPN

In [20]:
ESPN_sports_reference_dict = {}

In [21]:
ESPN_sports_reference_dict["Alabama-Birmingham"] = "UAB"

In [22]:
ESPN_sports_reference_dict["ETSU"] = "East Tennessee State"

In [23]:
ESPN_sports_reference_dict["UNC"] = "North Carolina"

In [24]:
ESPN_sports_reference_dict["St. Joseph's"] = "Saint Joseph's"

In [25]:
ESPN_sports_reference_dict["Pitt"] = "Pittsburgh"

In [26]:
ESPN_sports_reference_dict["UC-Irvine"] = "UC Irvine"

In [27]:
ESPN_sports_reference_dict["Cal State Fullerton"] = "CSU Fullerton"

In [28]:
ESPN_sports_reference_dict["College of Charleston"] = "Charleston"

In [29]:
ESPN_sports_reference_dict["Miami (FL)"] = "Miami"

In [30]:
ESPN_sports_reference_dict["Loyola (IL)"] = "Loyola Chicago"

In [31]:
ESPN_sports_reference_dict["Penn"] = "Pennsylvania"

In [32]:
ESPN_sports_reference_dict["Mount St. Mary's"] = "Mt. St. Mary's"

In [33]:
ESPN_sports_reference_dict["ETSU"] = "East Tennessee State"

In [34]:
ESPN_sports_reference_dict["UC-Davis"] = "UC Davis"

In [35]:
ESPN_sports_reference_dict["Hawaii"] = "Hawai'i"

In [36]:
ESPN_sports_reference_dict["Cal State Bakersfield"] = "CSU Bakersfield"

In [37]:
ESPN_sports_reference_dict["Albany (NY)"] = "Albany"

In [38]:
ESPN_sports_reference_dict["St. John's (NY)"] = "St. John's"

In [39]:
ESPN_sports_reference_dict["Alabama-Birmingham"] = "UAB"

In [40]:
ESPN_sports_reference_dict["Detroit"] = "Detroit Mercy"

In [41]:
ESPN_sports_reference_dict["UCSB"] = "UC Santa Barbara"

In [42]:
ESPN_sports_reference_dict["St. Peter's"] = "Saint Peter's"

In [43]:
ESPN_sports_reference_dict["Cal State Northridge"] = "CSU Northridge"

In [44]:
ESPN_sports_reference_dict["Cal State Fullerton"] = "CSU Fullerton"

In [45]:
ESPN_sports_reference_dict["Texas-Arlington"] = "UT Arlington"

In [46]:
ESPN_sports_reference_dict["Texas A&M-Corpus Christi"] = "Texas A&M-CC"

In [47]:
ESPN_sports_reference_dict["Southeastern Louisiana"] = "SE Louisiana"

In [48]:
ESPN_sports_reference_dict["McNeese State"] = "McNeese"

In [49]:
ESPN_sports_reference_dict["Prairie View"] = "Prairie View A&M"

In [50]:
ESPN_sports_reference_dict["Nicholls State"] = "Nicholls"

In [51]:
ESPN_sports_reference_dict["San Jose State"] = "San José St"

In [52]:
ESPN_sports_reference_dict["Louisiana-Monroe"] = "UL Monroe"

In [53]:
ESPN_sports_reference_dict["Saint Francis (PA)"] = "St. Francis (PA)"

In [54]:
ESPN_sports_reference_dict["LIU"] = "Long Island University"

In [55]:
ESPN_sports_reference_dict["Southeast Missouri State"] = "SE Missouri St"

# Compiles all functions from above

In [56]:
def really_everything (year):
    #getting sports reference stuff
    newdf2=everything ('https://www.sports-reference.com/cbb/postseason/'+f'{year}'+'-ncaa.html')
    the_year = []
    #print('before the for loop')
    for thing in newdf2['first_team']:
        the_year.append(year)
    newdf2['year']=the_year
    #print('right before new loop')
    
    

    
    #getting ESPN stuff
    df2=merge_espn (year)
    #print('finished espn')
    
    #merging
    possibly2 = pd.merge(newdf2,df2, left_on = 'name', right_on = 'Name')
    possibly2 = possibly2.sort_values(by ='index' )
    the_values=find_missing_values (list(possibly2['index']))
    for thing in the_values:
        #print(thing)
        the_name=list(newdf2[(newdf2['index']==thing)]['name'])[0]
        ESPN_name=ESPN_sports_reference_dict[f"{the_name}"]
        possibly2=add_to_df (ESPN_name, thing,possibly2, df2, newdf2)
    
    
    other_team_seed = []
    i = 0
    for thing in list(possibly2['team_rank']):
        if thing == list(possibly2['second_team'])[i]:
            other_team_seed.append(list(possibly2['first_team'])[i])
        elif thing == list(possibly2['first_team'])[i]:
            other_team_seed.append(list(possibly2['second_team'])[i])
        i+=1
    #print('made it here')
    possibly2['other_team_seed']=other_team_seed
    #print('finished sports reference')
    
    
    possibly2.drop(['first_team','second_team'], axis = 1, inplace = True)
    possibly2.rename(columns={'team_rank':'team_seed'}, inplace = True)
    possibly2 = possibly2.sort_values(by ='index' )

        
    return possibly2

# Grabbing the advanced stats

In [57]:
def get_advanced_stats (year):
    page = requests.get('https://www.sports-reference.com/cbb/seasons/'+f'{year}'+'-advanced-school-stats.html')
    soup = BS(page.content, 'html.parser')
    idk = soup.select('tbody')
    
    name_adv=[]
    SRS_adv = []
    SOS_adv = []
    eFG_perc_adv = []
    FTr_adv = []
    
    for thing in idk[0].select('tr'):
        try:

            #school name
            name=thing.select('td[data-stat="school_name"]')[0].select('a')[0].string
            

            #SRS
            SRS=float(thing.select('td[data-stat="srs"]')[0].string)
            

            #SOS
            SOS = float(thing.select('td[data-stat="sos"]')[0].string)
            

            #eFG%
            eFG = float(thing.select('td[data-stat="efg_pct"]')[0].string)
            

            #FTr
            FTr = float(thing.select('td[data-stat="fta_per_fga_pct"]')[0].string)
            
            
            name_adv.append(name)
            SRS_adv.append(SRS)
            SOS_adv.append(SOS)
            eFG_perc_adv.append(eFG)
            FTr_adv.append(FTr)
            
        except:
            continue
    
    the_year = []
    for thing in FTr_adv:
        the_year.append(year)
    
    D = {'name':name_adv,'SRS_adv':SRS_adv,'SOS_adv':SOS_adv,'eFG_perc_adv':eFG_perc_adv,'year_adv':the_year}
    adv_stats = pd.DataFrame(data=D)
    
    return adv_stats
        
        

# Making internal dictionary for sports reference to connect their march madness data and
#commented out code is for finding the names that need a dictionary for merging

In [58]:
#df is all_data_not_advanced
# team_names=list(df['name'].unique())
# team_names = {'name':(team_names)}
# team_names = pd.DataFrame(data=team_names)

In [59]:
# adv_team_names=list(adv_stats['name'])
# adv_team_names = {'name':(adv_team_names)}
# adv_team_names = pd.DataFrame(data=adv_team_names)

In [60]:
# please=pd.merge(adv_team_names,team_names, on = 'name')

In [61]:
# merged_names=list(please['name'])

In [62]:
# need_dictionary_names = []
# for name in list(team_names['name']):
#     if name in merged_names:
#         continue
#     else:
#         need_dictionary_names.append(name)

In [63]:
# need_dictionary_names

In [64]:
#march madness in the brackets and the adv stats name on the right side of the equal side
sports_ref_dict = {}

In [65]:
sports_ref_dict["SMU"]= "Southern Methodist"
sports_ref_dict["USC"]= "Southern California"
sports_ref_dict["Pitt"]= "Pittsburgh"
sports_ref_dict["Penn"]= "Pennsylvania"
sports_ref_dict["LSU"]= "Louisiana State"
sports_ref_dict["UNC"]= "North Carolina"
sports_ref_dict["UNLV"]= "Nevada-Las Vegas"
sports_ref_dict["UTEP"]= "Texas-El Paso"
sports_ref_dict["NC State"]= "North Carolina State"
sports_ref_dict["VCU"]= "Virginia Commonwealth"
sports_ref_dict["St. Joseph's"]= "Saint Joseph's"
sports_ref_dict["TCU"]= "Texas Christian"
sports_ref_dict["BYU"]= "Brigham Young"
sports_ref_dict["UTSA"]= "Texas-San Antonio"
sports_ref_dict["UCSB"]= "UC-Santa Barbara"
sports_ref_dict["ETSU"]= "East Tennessee State"
sports_ref_dict["Saint Mary's"]="Saint Mary's (CA)"
sports_ref_dict["UConn"]="Connecticut"
sports_ref_dict["California"]="University of California"
sports_ref_dict["Southern Miss"]="Southern Mississippi"
sports_ref_dict["St. Peter's"]="Saint Peter's"
sports_ref_dict["UMass"]="Massachusetts"
sports_ref_dict["Long Beach State"]="Cal State Long Beach"
sports_ref_dict["UCF"]="Central Florida"
sports_ref_dict["UNC Greensboro"]="North Carolina-Greensboro"
sports_ref_dict["LIU"]="Long Island University"
sports_ref_dict["Ole Miss"]="Mississippi"
sports_ref_dict["UIC"]="Illinois-Chicago"
sports_ref_dict["Detroit"]="Detroit Mercy"
sports_ref_dict["Central Connecticut"]="Central Connecticut State"
sports_ref_dict["UNC Wilmington"]="North Carolina-Wilmington"
sports_ref_dict["UNC Asheville"]="North Carolina-Asheville"
sports_ref_dict["UMBC"]="Maryland-Baltimore County"

# Function for merging with sports reference adv data and uses created dictionary

In [66]:
def merge_sr (df1,adv_stats,year):
    merged_df=pd.merge(df1[(df1['year_x']==year)], adv_stats[(adv_stats['year_adv']==year)],
                 how = 'inner', left_on='names_for_merging_sr', right_on = 'name')
    return merged_df

# addind in opponent data and subtraction of opponent data to adv_stats

In [67]:
#pass through the dataframe you want to add to and the column name
#it will create a new column with the columnname_opponent
def get_opponent_data (df, column_name):
    first_data = []
    second_data = []
    i=0
    x = 0
    for thing in df[f'{column_name}']:
        if i ==0:
            first_data.append(df[f'{column_name}'].iloc[x+1])
            i+=1
            x+=1
        elif i==1:
            second_data.append(df[f'{column_name}'].iloc[x-1])
            i=0
            x+=1
    
    i=0
    x=0
    opponent_data = []
    for thing in first_data+second_data:
        if i==0:
            opponent_data.append(first_data[x])
            i+=1
        elif i==1:
            opponent_data.append(second_data[x])
            x+=1
            i=0
    print('made it here')        
    df[f'{column_name}'+'_opponent']=opponent_data

In [68]:
def subtracted_stats  (df,column):
    i = 0
    subtracted = []
    for thing in list(df[f'{column}']):
        difference=list(df[f'{column}'])[i]-list(df[f'{column}'+'_opponent'])[i]
        subtracted.append(difference)
        i+=1
    
    df[f'{column}'+'_subtracted']=subtracted

# Insert the rounds back in

In [69]:
#make sure at bottom to change number in function when reassigning the round_ to 0:number in dataframe
def insert_rounds (df,column):
    round_=[]
    i=0
    for thing in list(column):
        if i== 62:
            round_.append(6)
            round_.append(6)
            i=0
        elif 0<=i<=7 or 15<=i<=22 or 30<=i<=37 or 45<=i<=52:
            round_.append(1)
            round_.append(1)
            i+=1
        elif 8<=i<=11 or 23<=i<=26 or 38<=i<=41 or 53<=i<=56:
            round_.append(2)
            round_.append(2)
            i+=1
        elif 12<=i<=13 or 27<=i<=28 or 42<=i<=43 or 57<=i<=58:
            round_.append(3)
            round_.append(3)
            i+=1
        elif i==14 or i==29 or i==44 or i==59:
            round_.append(4)
            round_.append(4)
            i+=1
        elif i == 60 or i == 61:
            round_.append(5)
            round_.append(5)
            i+=1
    round_ = round_[0:126]
    return round_

# Getting Kenpom Ratings and wins/losses from regular season

In [70]:
def get_kenpom_wins_losses (year):
    page = requests.get('https://kenpom.com/index.php?y='+f'{year}')
    soup = BS(page.content, 'html.parser')
    idk = soup.select('tr')


    names = []
    wins_losses = []
    kenpom_adjem = []
    for thing in idk:
        try:
            names.append(thing.select('a[href^="team.php?team="]')[0].string)
            wins_losses.append(thing.select('td[class="wl"]')[0].text)
            kenpom_adjem.append(float(thing.select('td')[4].text))          
        except:
            continue
            
    wins = []
    losses = []
    for thing in wins_losses:
        i=0
        for num in thing:
            if num !='-':
                i+=1
            elif num =='-':
                wins.append(float(thing[0:i]))
                losses.append(float(thing[i+1::]))
                break
    year_kenpom = []
    for thing in wins:
        year_kenpom.append(int(year))
            
    D = {'name_ken':names,'wins':wins,'losses':losses,'year_kenpom':year_kenpom,'kenpom_adjem':kenpom_adjem}
    kenpom_wins_losses = pd.DataFrame(data=D)
    return kenpom_wins_losses

# Making a dictionary for KenPom names to merge with all other data

In [73]:
#to figure out which names you need to make a dictionary for
need_dictionary_names = []
for thing in list(merged_df['Name']):
    if thing in list(ken_pom['name_ken']):
        continue
    else:
        need_dictionary_names.append(thing)

In [74]:
#these are the names you need to make a dictionary for
set(need_dictionary_names)

In [75]:
#Name in the brackets and the kenpom name on the right side of the equal side
kenpom_dict = {}
kenpom_dict["Alabama State"]="Alabama St."
kenpom_dict["Alcorn State"]="Alcorn St."
kenpom_dict["Arkansas-Pine Bluff"]="Arkansas Pine Bluff"
kenpom_dict["CSU Bakersfield"]="Cal St. Bakersfield"
kenpom_dict["CSU Fullerton"]="Cal St. Fullerton"
kenpom_dict["CSU Northridge"]="Cal St. Northridge"
kenpom_dict["Detroit Mercy"]="Detroit"
kenpom_dict["Florida International"]="FIU"
kenpom_dict["Gardner-Webb"]="Gardner Webb"
kenpom_dict["Hawai'i"]="Hawaii"
kenpom_dict["Long Island University"]="LIU"
kenpom_dict["Loyola (MD)"]="Loyola MD"
kenpom_dict["McNeese"]="McNeese St."
kenpom_dict["Miami"]="Miami FL"
kenpom_dict["Miami (OH)"]="Miami OH"
kenpom_dict["Mt. St. Mary's"]="Mount St. Mary's"
kenpom_dict["NC State"]="N.C. State"
kenpom_dict["Nicholls"]="Nicholls St."
kenpom_dict["Ole Miss"]="Mississippi"
kenpom_dict["Pennsylvania"]="Penn"
kenpom_dict["SE Louisiana"]="Southeastern Louisiana"
kenpom_dict["SE Missouri St"]="Southeast Missouri St."
kenpom_dict["San José St"]="San Jose St."
kenpom_dict["Texas A&M-CC"]="Texas A&M Corpus Chris"
kenpom_dict["UConn"]="Connecticut"
kenpom_dict["UIC"]="Illinois Chicago"
kenpom_dict["UL Monroe"]="Louisiana Monroe"
kenpom_dict["UMass"]="Massachusetts"







In [142]:
#a lot of the names I just needed to go from state --> st.
#below code takes care of this in mass
for name in need_dictionary_names:
    if name in list(kenpom_dict.keys()):
        continue
    else:
        kenpom_dict[f'{name}']=name[0:len(name)-3]+'.'

In [76]:
#flip the keys and values
kenpom_dict2 = {y:x for x,y in kenpom_dict.items()}


In [77]:
kenpom_dict2["Troy St."]="Troy"

In [78]:
kenpom_dict2["Louisiana Lafayette"]="Louisiana"

In [79]:
kenpom_dict2["LIU Brooklyn"]="Long Island University"

In [80]:
kenpom_dict2["Arkansas Little Rock"]="Little Rock"

In [81]:
kenpom_dict2["College of Charleston"]="Charleston"

# Function for merging Ken_Pom with everything else

In [155]:
def merge_sr (df1,ken_pom,year):
    merged_df=pd.merge(df1[(df1['year_x']==year)], ken_pom[(ken_pom['year_kenpom']==year)],
                 how = 'inner', left_on='Name', right_on = 'names_for_merging_ken')
    return merged_df

# Data collection - using all the functions to collect the data

In [ ]:
#below gets regular stats from espn and regular seeding sports reference stuff

In [87]:
all_data_not_advanced=really_everything (2003)

https://www.sports-reference.com/cbb/postseason/2003-ncaa.html


In [ ]:
years = []
beg = 1986
for num in range(1,35):
    years.append(beg)
    beg+=1

In [ ]:
for year in years:
    data=really_everything(year)
    all_data_not_advanced=all_data_not_advanced.append(data, ignore_index=True)

In [ ]:
i=0
index_ = []
for thing in all_data_not_advanced['name']:
    index_.append(i)
    i+=1
    
all_data_not_advanced['index_everything']=index_

In [ ]:
#below gets advanced stats

In [ ]:
adv_stats=get_advanced_stats (2005)

In [ ]:
years = []
first = 1994
for num in range(0,26):
    years.append(first)
    first+=1

In [ ]:
for year in years:
    print(year)
    adv_stat=get_advanced_stats (year)
    adv_stats=adv_stats.append(adv_stat, ignore_index=True)

at this point make sure the dictionary is good in 
"making internal dictionary for sports reference to connect their march madness data"
Uncomment out the stuff to double check that the dictionary has everything

In [ ]:
#below uses dictionary and adds in a column into the original sports reference ESPN stuff to merge on

In [ ]:
names_for_merging_sr = []
for name in list(all_data_not_advanced['name']):
    if name in sports_ref_dict.keys():
        names_for_merging_sr.append(sports_ref_dict[f'{name}'])
    else:
        names_for_merging_sr.append(name)

In [ ]:
all_data_not_advanced['names_for_merging_sr']=names_for_merging_sr

In [ ]:
#now below actually merges everything

In [ ]:
years = []
first = 1993
for num in range(0,27):
    years.append(first)
    first+=1

In [ ]:
years = [2005]

In [ ]:
i=0
for year in years:
    if i==0:
        merged_df=merge_sr (all_data_not_advanced,adv_stats,year)
        merged_df = merged_df.sort_values(by ='index_everything' )
        i=1
    else:
        merged=merge_sr (all_data_not_advanced,adv_stats,year)
        merged_df=merged_df.append(merged, ignore_index=True)
        merged_df = merged_df.sort_values(by ='index_everything' )

In [ ]:
#gets opponent and subtracted stuff for advanced stats

In [ ]:
columns = ['SRS_adv', 'SOS_adv', 'eFG_perc_adv']

In [ ]:
for column in columns:
    get_opponent_data (merged_df, column)

In [ ]:
for thing in columns:
    subtracted_stats(merged_df,thing)

In [ ]:
#insert round the game was played in into dataframe

In [ ]:
#go to the function insert rounds and make sure the round_ = round_[0:3402]
#right now it says 0:3402, but make sure the number after 0: is the length of your dataframe
blah=insert_rounds (merged_df,merged_df['point_difference'])

In [ ]:
merged_df['round']=blah

In [ ]:
merged_df.columns

In [ ]:
opponent_columns = ['BLKPG',
       'STPG', 'APG', 'AST_TO', '_3pm', '_3pa', '_2pm', '_2pa', '_2p_perc',
       'FG_perc', '_3p_perc', 'FT_perc', 'ORPG', 'DRPG', 'RPG',
       'PTS']

In [ ]:
for column in opponent_columns:
    get_opponent_data (merged_df, f'{column}')

In [ ]:
for thing in opponent_columns:
    subtracted_stats(merged_df,thing)

In [ ]:
#below is getting the kenpom stuff

In [ ]:
ken_pom=get_kenpom_wins_losses (2005)

In [ ]:
years = []
first = 2003
for num in range(0,17):
    years.append(first)
    first+=1

In [ ]:
for year in years:
    KP=get_kenpom_wins_losses (year)
    ken_pom=ken_pom.append(KP, ignore_index=True)

In [ ]:
#below inserts another column with a name to merge ken_pom stuff with everything else
#Go to 'Making a dictionary for KenPom names to merge with all other data' to run code to make sure
#you have all the names in the dictionary to merge

In [ ]:
names_for_merging_ken = []
for name in list(ken_pom['name_ken']):
    if name in kenpom_dict2.keys():
        names_for_merging_ken.append(kenpom_dict2[f'{name}'])
    else:
        names_for_merging_ken.append(name)

In [ ]:
ken_pom['names_for_merging_ken']=names_for_merging_ken

In [ ]:
#below merges ken_pom with everything else

In [ ]:
years = []
first = 2002
for num in range(0,18):
    years.append(first)
    first+=1

In [ ]:
i=0
for year in years:
    if i==0:
        merged_df1=merge_sr (merged_df,ken_pom,year)
        merged_df1 = merged_df1.sort_values(by ='index_everything' )
        i=1
    else:
        merged=merge_sr (merged_df,ken_pom,year)
        merged_df1=merged_df1.append(merged, ignore_index=True)
        merged_df1 = merged_df1.sort_values(by ='index_everything' )

In [ ]:
#below gets opponent data and difference between team and opponent for ken_pom stuff and wins/losses

In [ ]:
opponent_columns = ['wins', 'losses', 'kenpom_adjem']

In [ ]:
for column in opponent_columns:
    get_opponent_data (merged_df1, f'{column}')

In [ ]:
for thing in opponent_columns:
    subtracted_stats(merged_df1,thing)